<a href="https://colab.research.google.com/github/Vincenzo-Miracula/TallerPratico/blob/main/EscuelaGobierno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# librerie

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
tqdm.pandas()
import re

# dataframe e analisi dati

In [ ]:
df = pd.read_excel('')

In [ ]:
df

# sentiment-emotion

In [ ]:
def clean(text):
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticon
                           u"\U0001F300-\U0001F5FF"  # simboli & pittogrammi
                           u"\U0001F680-\U0001F6FF"  # trasporti & simboli mappe
                           u"\U0001F1E0-\U0001F1FF"  # bandiere (emoji)
                           u"\U00002500-\U00002BEF"  # caratteri cinesi comuni
                           u"\U00002702-\U000027B0"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           u"\u2640-\u2642"
                           u"\u2600-\u2B55"
                           u"\u200d"
                           u"\u23cf"
                           u"\u23e9"
                           u"\u231a"
                           u"\ufe0f"  # combinazione di caratteri
                           u"\u3030"
                           "]+", flags=re.UNICODE)
  text = emoji_pattern.sub(r'', text)
  text = re.sub(r'http\S+|www\S+|https\S+', '', text)
  text = re.sub(r'\w+#\w+', '', text)
  text = re.sub(r'@\w+\s*', '', text)
  text = re.sub(r'RT @\w+:', '', text)

  text = text.lower()
  return text.strip()

In [ ]:
df['pulito'] = df['Open-Ended Response'].progress_apply(clean)

In [ ]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
candidate_labels = ["Positive", "Neutral", "Negative", "AlmostPositive", "AlmostNegative"]

In [ ]:
text = 'Hola a todo el mundo'
classifier(text, candidate_labels, multi_label=False)

In [ ]:
def sentiment(text):
  result = classifier(text, candidate_labels, multi_label=False)
  return result['labels'][0]

In [ ]:
df['SentimentText'] = df['es_text_pulito'].progress_apply(sentiment)

# scraping

In [ ]:
#Esta línea importa la clase BeautifulSoup de la biblioteca bs4 y le asigna un alias 'bs'
from bs4 import BeautifulSoup as bs
#Esta línea importa la biblioteca requests, que se utiliza para realizar solicitudes HTTP a páginas web
import requests
# Importar la biblioteca 'time' para manejar el tiempo en el script.
import time

In [ ]:
# Se establece un encabezado HTTP para simular un navegador web (User-Agent) en la solicitud.
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
# Se hace una solicitud GET a la URL proporcionada con el encabezado especificado.
url = requests.get('https://www.plenainclusion.org/noticias/', headers=headers)
# Se utiliza BeautifulSoup para analizar el contenido de la respuesta XML.
soup = bs(url.content, "html.parser")
badges = soup.find_all('div', {'class': 'elementor-post__badge'})
titulos = soup.find_all("div", {'class':"elementor-post__text"})
textos = soup.find_all("div", {'class':"elementor-post__excerpt"})

In [ ]:
data = []
for badge, titulo, texto in zip(badges, titulos, textos):
  badge = badge.text.strip()
  titulo = titulo.text.strip()
  texto = texto.text.strip()
  data.append([badge, titulo, texto])

In [ ]:
data #pd.DataFrame(data, columns=['Tag', 'Titulo', 'Texto'])

In [ ]:
url = "https://www.plenainclusion.org/noticias/?sf_paged={}"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
start_page=1
end_page=15

data = []

for page_num in tqdm(range(start_page, end_page + 1)):
  url_pagina = url.format(page_num)
  response = requests.get(url_pagina, headers)
  soup = bs(response.content, "html.parser")

  badges = soup.find_all('div', {'class': 'elementor-post__badge'})
  titulos = soup.find_all("div", {'class':"elementor-post__text"})
  textos = soup.find_all("div", {'class':"elementor-post__excerpt"})
  for badge, titulo, texto in zip(badges, titulos, textos):
    badge = badge.text.strip()
    titulo = titulo.text.strip()
    texto = texto.text.strip()
    data.append([badge, titulo, texto])

In [ ]:
data #pd.DataFrame(data, columns=['Tag', 'Titulo', 'Texto'])